In [1]:
import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader 
import sys 
sys.path.append('../')

from data_yours import ALS_50K

In [2]:
# get sample 
ds = ALS_50K(num_points=2048)
sample = ds[25542]

In [3]:
# get xyz 
sample = sample[0]
sample.shape

(2048, 3)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
sample = torch.tensor(sample, dtype=torch.float32).to(device)  # add batch dimension

## For a single point cloud

## kNN Clustering function 

In [6]:
from kmeans_pytorch import kmeans

In [ ]:
def get_tree_center_xy(pc): 
    return torch.mean(pc[:,:2], dim=0)
get_tree_center_xy(sample)

In [36]:
def initialize_clusters(pc, device, k=100):
    c_idx, c_centroids = kmeans(
        X=pc,
        num_clusters=k,
        distance='euclidean',
        device=device
    )
    return c_idx, c_centroids
cl_idx, cl_centroids = initialize_clusters(sample, device, k=100)
cl_idx.shape, cl_centroids.shape

running k-means on cuda..


[running kmeans]: 11it [00:00, 57.28it/s, center_shift=0.000000, iteration=11, tol=0.000100]


(torch.Size([2048]), torch.Size([100, 3]))

In [37]:
def get_clusters(pc, c_idx):
    """
    Splits the point cloud (pc) into clusters according to c_idx.
    Returns a list of tensors, one per cluster.
    """
    return [pc[c_idx == i] for i in range(c_idx.max() + 1)]
clusters = get_clusters(sample, cl_idx)
len(clusters), clusters[0].shape

(100, torch.Size([16, 3]))

In [38]:
def compute_pca(points):
    mean = points.mean(dim=0)
    X = points - mean
    eigvals, eigvecs = torch.linalg.eigh(torch.cov(X.T))
    idx = torch.argsort(eigvals, descending=True)
    eigvecs = eigvecs[:, idx]
    direction = eigvecs[:, 0]  # 1st principal component
    normal = eigvecs[:, 1]     # 2nd principal component
    return mean, direction, normal
compute_pca(clusters[0])

(tensor([ 0.0554,  0.1117, -0.2008], device='cuda:0'),
 tensor([-0.2600, -0.0675,  0.9633], device='cuda:0'),
 tensor([-0.5133, -0.8353, -0.1970], device='cuda:0'))

In [39]:
def cluster_residuals(cluster):
    centroid, direction, normal = compute_pca(cluster)
    vec = cluster - centroid 
    normal = normal / torch.norm(normal) 
    distances = torch.abs(torch.matmul(vec, normal))
    return torch.sum(distances)
cluster_residuals(clusters[0])


tensor(0.5534, device='cuda:0')

In [46]:
def merged_residual_diff_percent(c1, c2):
    r_prior = cluster_residuals(c1) + cluster_residuals(c2)
    r_post = cluster_residuals(merge_clusters(c1, c2))
    return (r_post - r_prior) / r_prior * 100.0 

merged_residual_diff_percent(c1, c2)

tensor(70.6257, device='cuda:0')

In [41]:
def distance_pairs(centroids): 
    dmatrix = torch.cdist(centroids, centroids) 
    dmatrix = dmatrix.fill_diagonal_(float('inf'))
    tril_indices = torch.tril_indices(dmatrix.size(0), dmatrix.size(1), offset=-1)
    distances = dmatrix[tril_indices[0], tril_indices[1]]
    sorted_indices = distances.argsort()
    sorted_pairs = torch.stack((tril_indices[0][sorted_indices], tril_indices[1][sorted_indices]), dim=1)
    sorted_distances = distances[sorted_indices]
    return sorted_pairs, sorted_distances

distance_pairs(cl_centroids)

(tensor([[89, 67],
         [32, 30],
         [79, 15],
         ...,
         [60, 23],
         [97, 23],
         [40, 23]]),
 tensor([0.0591, 0.0627, 0.0654,  ..., 1.7361, 1.7436, 1.7535]))

In [42]:
def pairs_angle(c1, c2): 
    c1_mean, c1_direction, c1_normal = compute_pca(c1)
    c2_mean, c2_direction, c2_normal = compute_pca(c2)
    angle = torch.acos(torch.dot(c1_normal, c2_normal) / (torch.norm(c1_normal) * torch.norm(c2_normal)))
    return angle

c1 = clusters[89]
c2 = clusters[67]
pairs_angle(c1, c2)

tensor(0.6252, device='cuda:0')

In [35]:
def merge_clusters(c1, c2):
    merged = torch.cat((c1, c2), dim=0)
    return merged

print(c1.shape, c2.shape)
merged = merge_clusters(c1, c2)
print(merged.shape)

torch.Size([8, 3]) torch.Size([6, 3])
torch.Size([14, 3])


In [49]:
def compute_threshold(rdpercents): 
    return torch.quantile(rdpercents, 0.75)

In [58]:
# all the way stage one test 
c_idx, c_centroids = initialize_clusters(sample, device, k=100)
clusters = get_clusters(sample, cl_idx)
pairs, distances = distance_pairs(c_centroids)

rdpercent = [] 
for i, (c1_idx, c2_idx) in enumerate(pairs):
    c1 = clusters[c1_idx]
    c2 = clusters[c2_idx]
    rdpercent.append(merged_residual_diff_percent(c1, c2))
rpercent = torch.tensor(rdpercent, dtype=torch.float32).to(device)

running k-means on cuda..


[running kmeans]: 0it [00:00, ?it/s, center_shift=12.905282, iteration=1, tol=0.000100]

[running kmeans]: 13it [00:00, 51.74it/s, center_shift=0.000000, iteration=13, tol=0.000100]


In [60]:
t = compute_threshold(rpercent)
t

tensor(69.5705, device='cuda:0')

In [13]:
# get all clusters 
clusters_xyz = [sample[c_idx == i][:, :3] for i in range(num_clusters)]

NameError: name 'num_clusters' is not defined

In [47]:
# compute distances between centroids 
distance_matrix = torch.cdist(c_centroids, c_centroids)
distance_matrix = distance_matrix.fill_diagonal_(float('inf'))
tril_indices = torch.tril_indices(distance_matrix.size(0), distance_matrix.size(1), offset=-1)
distances = distance_matrix[tril_indices[0], tril_indices[1]]

In [51]:
sorted_distances, sorted_indices = torch.sort(distances)
pairs = torch.stack((tril_indices[0][sorted_indices], tril_indices[1][sorted_indices]), dim=1)

In [57]:
c_centroids = c_centroids.cuda()

In [58]:
# normalize clusters 
clusters_xyz = [c - c_centroids[i] for i, c in enumerate(clusters_xyz)]

In [89]:
# for each cluster in clusters_xyz, compute the PCA
cov_matrices = [torch.cov(cluster.T) for cluster in clusters_xyz]

In [90]:
eigen = [torch.linalg.eigh(cov_matrix) for cov_matrix in cov_matrices]

In [91]:
eigen_idx = [torch.argsort(eigen[i].eigenvalues, descending=True) for i in range(num_clusters)]

In [92]:
eigenvalues_sorted = [eigen[i].eigenvalues[eigen_idx[i]] for i in range(num_clusters)]

In [93]:
eigenvectors_sorted = [eigen[i].eigenvectors[:, eigen_idx[i]] for i in range(num_clusters)]

In [ ]:
# Function to get normal, direction 

In [94]:
eigenvalues_sorted, eigenvectors_sorted = [], []

for cov in cov_matrices:
    vals, vecs = torch.linalg.eigh(cov)
    idx = torch.argsort(vals, descending=True)
    eigenvalues_sorted.append(vals[idx])
    eigenvectors_sorted.append(vecs[:, idx])

In [95]:
# extract normals of the cluster -> the second eigenvector 
direct_vectors = [eigenvectors_sorted[i][:, 0] for i in range(num_clusters)]
normals = [eigenvectors_sorted[i][:, 1] for i in range(num_clusters)]

In [99]:
# calculate the angle between the best-fit planes of each pair of clusters 
angles = [] 
for i in range(len(pairs)):
    idx1, idx2 = pairs[i, 0], pairs[i, 1]
    normal1 = normals[idx1]
    normal2 = normals[idx2]
    angle = torch.acos(torch.dot(normal1, normal2) / (torch.norm(normal1) * torch.norm(normal2)))
    angles.append(angle.item())

In [105]:
angles = torch.tensor(angles)
angles_sorted, angles_indices = torch.sort(angles, descending=False)

/tmp/ipykernel_1466842/1225822824.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  angles = torch.tensor(angles)


In [124]:
def calculate_residuals(clusters_xyz, c_centroids, normals):
    """
    Calculate the residuals for each cluster based on the distance from points to the plane defined by the centroid and normal.
    """
    residuals = []
    for i in range(num_clusters):
        cluster = clusters_xyz[i]
        centroid = c_centroids[i]
        normal = normals[i]
        # calculate the distance from each point in the cluster to the plane defined by the centroid and normal
        vec = cluster - centroid 
        normal = normal / torch.norm(normal) 
        distances = torch.abs(torch.matmul(vec, normal))
        residual = torch.sum(distances)
        residuals.append(residual.item())
    return torch.tensor(residuals) 

In [125]:
res_prior = calculate_residuals(clusters_xyz, c_centroids, normals)

In [128]:
# function to merge pairs of clusters 
def merge_clusters(clusters_xyz, pairs): 
    """
    Merge clusters based on the pairs provided.
    """
    merged_clusters = []
    for i in range(len(pairs)):
        idx1, idx2 = pairs[i, 0], pairs[i, 1]
        # Concatenate the clusters along the first dimension
        merged_cluster = torch.cat([clusters_xyz[idx1], clusters_xyz[idx2]], dim=0)
        merged_clusters.append(merged_cluster)
    
    return merged_clusters

In [129]:
merged_pairs = merge_clusters(clusters_xyz, pairs)

In [132]:
post_merge_centroids = torch.stack([torch.mean(cluster, dim=0) for cluster in merged_pairs])    

In [ ]:
post_merge_normals = 

In [162]:
# paris 
cluster_pairs_idx = sorted_cluster_pairs[:, 0:2].int()

In [163]:
pair_idx = cluster_pairs_idx[0, :2]

In [166]:
normal_x = normals[pair_idx[0]]
normal_y = normals[pair_idx[1]]

In [169]:
def calculate_angle(normal_x, normal_y): 
    return torch.acos(torch.dot(normal_x, normal_y) / (torch.norm(normal_x) * torch.norm(normal_y)))

In [173]:
def calculate_angles_per_pair(cluster_pairs_idx, normals): 
    angles = [] 
    for pair_idx in cluster_pairs_idx: 
        normal_x = normals[pair_idx[0]]
        normal_y = normals[pair_idx[1]]
        angle = calculate_angle(normal_x, normal_y)
        angles.append(angle.item())
    angles = torch.tensor(angles, dtype=torch.float32)
    return angles

In [174]:
cluster_angles = calculate_angles_per_pair(cluster_pairs_idx, normals)

In [177]:
cluster_angles_sorted_idx = torch.argsort(cluster_angles, descending=True)

In [ ]:
def calculate_threshold

In [178]:
cluster_angles_sorted_idx

tensor([79, 57, 76,  6,  7, 10, 52, 72, 75,  5, 43, 44, 20, 19, 34, 35, 84, 85,
         4,  3, 49, 48, 80, 78, 62, 59, 58, 90,  9,  8, 83, 99, 71, 36, 37, 81,
        26, 27, 56, 55, 69, 15, 16, 86, 87, 91, 23, 17, 18, 64, 65, 66, 77, 39,
        38, 47, 42, 70, 88, 14, 74, 22, 21, 24, 68, 29, 30, 92, 93, 13, 96, 95,
        94, 97, 98, 25, 54, 53, 67, 50, 51,  2, 73, 40, 41, 33, 11, 12, 28, 89,
         1,  0, 32, 31, 45, 46, 60, 61, 82, 63])